# 📢 Function Calling (❌ Currently broken for notebooks; use C# project instead)

The most powerful feature of chat completion is the ability to call functions from the model. This allows you to create a chat bot that can interact with your existing code, making it possible to automate business processes, create code snippets, and more.

The process seems complex, but much of the work is handled by the Semantic Kernel framework and the LLM. Here's what it looks like:
![](https://learn.microsoft.com/en-us/semantic-kernel/media/functioncalling.png)

Let's start in the usual way by setting up our environment and then we'll dig into how to make function calling work.

## 🛠️ Setup

In [ ]:
// Install the required packages
#r "nuget: Microsoft.Extensions.AI.Ollama, 9.1.0-preview.1.25064.3"
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.SemanticKernel.Connectors.Ollama, 1.33.0-alpha"
#r "nuget: OllamaSharp"

// Setup the URL
using System.Net.Sockets;

var ollamaApiUrl = "http://localhost:11434";
try {
  var tcp = new TcpClient();
  tcp.SendTimeout = 100;
  tcp.Connect("host.docker.internal", 11434);
  ollamaApiUrl = "host.docker.internal";
} catch { }

// Setup the model we want to use
var languageModel = "llama3.2:3b";

## 🧩 Create a plugin

To start calling functions we need to create a plugin so the Semantic Framework can power up our language model. Let's create a very basic pizza ordering plugin. While a real one would be more complex this will serve as a simple example.

In [ ]:
using System.ComponentModel;
using Microsoft.SemanticKernel;

public enum PizzaSize
{
  Small,
  Medium,
  Large
}

public class PizzaOrder
{
  public PizzaSize Size { get; set; }
  public List<string> Toppings { get; set; }
  public int Quantity { get; set; }
  public string SpecialInstructions { get; set; }
}

public class OrderPizzaPlugin
{
  [KernelFunction("order_pizza")]
  [Description("Order a pizza with the given size and toppings")]
  public async Task<PizzaOrder> OrderPizza(PizzaSize size, List<string> toppings, int Quantity, string SpecialInstructions) =>
    await Task.FromResult(new PizzaOrder
    {
      Size = size,
      Toppings = toppings,
      Quantity = Quantity,
      SpecialInstructions = SpecialInstructions
    });
}

## 🔌 Add the plugin to the kernel

Now that we have defined our plugin (a pizza ordering service), we need to add it into the kernel so it can be sent to the language model. Sending the information about our plugin to the model allows it to make use of the plugin.

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;

#pragma warning disable SKEXP0070

var kernelBuilder = Kernel.CreateBuilder()
  .AddOllamaChatCompletion(languageModel, new Uri(ollamaApiUrl));

kernelBuilder.Plugins.AddFromType<OrderPizzaPlugin>("OrderPizza");

var kernel = kernelBuilder.Build();

IChatCompletionService chatCompletion = kernel.GetRequiredService<IChatCompletionService>();
ChatHistory chatHistory = [
  new () { Role = AuthorRole.User, Content = "I'd like to order a pizza!" }
];

var chatResponse = await chatCompletion.GetChatMessageContentAsync(chatHistory, kernel: kernel);

Console.WriteLine(chatResponse);
chatHistory.Add(chatResponse);

## TODO: Use code in `semantic-kernel-tutorial/FunctionCalling.cs` to complete this when possible.

## 📚 Read More

[Semantic Kernel - Function calling](https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/function-calling)